## Calculating Impact Score

Calculating an impact score encompassing property tax change, median income change, # of new construction units, and # of zoning appeals or variances. The impact score will use data from 2015-2023 so it will measure impact on a 8 year timeline.

In [3]:
import pandas as pd
from pandas import json_normalize
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

### Property Tax Change 

In [ ]:
#property assessment data - had trouble with API so we're going csv route
prop = pd.read_csv("C:\Penn MCP\Fourth Semester\Practicum\data\prop.csv")

prop.head()

,parcel_number,year,market_value,taxable_land,taxable_building,exempt_land,exempt_building,objectid
0,11007810,2019,42600.0,0.0,0.0,42600.0,0.0,3145815343
1,11007810,2018,42600.0,0.0,0.0,42600.0,0.0,3145815344
2,11007810,2017,42600.0,0.0,0.0,42600.0,0.0,3145815345
3,11007810,2016,14900.0,0.0,0.0,14900.0,0.0,3145815346
4,11007810,2015,14900.0,0.0,0.0,14900.0,0.0,3145815347


In [26]:
prop_filtered = prop[(prop['year'] >= 2015) & (prop['year'] <= 2023)]

# Calculate total taxable value (sum of taxable land and building)
prop_filtered['taxable_value'] = prop_filtered['taxable_land'] + prop_filtered['taxable_building']

# Check if a property is tax-exempt (both taxable land and building are 0)
prop_filtered['is_exempt'] = prop_filtered['taxable_value'] == 0

# Philadelphia Real Estate Tax rates by year (from the official document)
tax_rates = {
    2015: 0.01340,   # 1.340%
    2016: 0.013998,  # 1.3998%
    2017: 0.013998,  # 1.3998%
    2018: 0.013998,  # 1.3998%
    2019: 0.013998,  # 1.3998%
    2020: 0.013998,  # 1.3998% (same as 2019 based on document)
    2021: 0.013998,  # 1.3998% (same as 2019 based on document)
    2022: 0.013998,  # 1.3998% (same as 2019 based on document)
    2023: 0.013998   # 1.3998% (same as 2019 based on document)
}

# Calculate property tax for all properties
prop_filtered['property_tax'] = prop_filtered.apply(
    lambda row: row['taxable_value'] * tax_rates.get(row['year'], 0), 
    axis=1
)

# Filter to get only taxable properties
taxable_prop = prop_filtered[~prop_filtered['is_exempt']]


C:\Users\cnmit\AppData\Local\Temp\ipykernel_29284\976477558.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_filtered['taxable_value'] = prop_filtered['taxable_land'] + prop_filtered['taxable_building']
C:\Users\cnmit\AppData\Local\Temp\ipykernel_29284\976477558.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop_filtered['is_exempt'] = prop_filtered['taxable_value'] == 0
C:\Users\cnmit\AppData\Local\Temp\ipykernel_29284\976477558.py:23: SettingWithCopyWarning: 
A value is trying to be set on a

In [27]:
taxable_prop.head()

,parcel_number,year,market_value,taxable_land,taxable_building,exempt_land,exempt_building,objectid,taxable_value,is_exempt,property_tax
7,11008200,2023,299100.0,59820.0,159280.0,0.0,80000.0,3145815350,219100.0,False,3066.9618
8,11008200,2022,283800.0,74356.0,164444.0,0.0,45000.0,3145815351,238800.0,False,3342.7224
9,11008200,2021,283800.0,74356.0,164444.0,0.0,45000.0,3145815352,238800.0,False,3342.7224
10,11008200,2020,283800.0,74356.0,209444.0,0.0,0.0,3145815353,283800.0,False,3972.6324
11,11008200,2019,261900.0,68617.0,193283.0,0.0,0.0,3145815354,261900.0,False,3666.0762
